In [9]:
import os
os.chdir("/local/crv/acanberk/folding-unfolding/src")
# os.system(". ./prepare.sh")
# os.system("./prepare.sh")
# os.system("python")
import numpy as np
import torch

def compute_intrinsics(fov, image_size):
    image_size = float(image_size)
    focal_length = (image_size / 2)\
        / np.tan((np.pi * fov / 180) / 2)
    return np.array([[focal_length, 0, image_size / 2],
                     [0, focal_length, image_size / 2],
                     [0, 0, 1]])

def compute_pose(pos, lookat, up=[0, 0, 1]):
    norm = np.linalg.norm
    if type(lookat) != np.array:
        lookat = np.array(lookat)
    if type(pos) != np.array:
        pos = np.array(pos)
    if type(up) != np.array:
        up = np.array(up)
    f = (lookat - pos)
    f = f/norm(f)
    u = up / norm(up)
    s = np.cross(f, u)
    s = s/norm(s)
    u = np.cross(s, f)
    view_matrix = [
        s[0], u[0], -f[0], 0,
        s[1], u[1], -f[1], 0,
        s[2], u[2], -f[2], 0,
        -np.dot(s, pos), -np.dot(u, pos), np.dot(f, pos), 1
    ]
    view_matrix = np.array(view_matrix).reshape(4, 4).T
    pose_matrix = np.linalg.inv(view_matrix)
    pose_matrix[:, 1:3] = -pose_matrix[:, 1:3]
    return pose_matrix



def pixel_to_3d(depth_im, x, y,
                pose_matrix,
                fov=39.5978,
                depth_scale=1):
    intrinsics_matrix = compute_intrinsics(fov, depth_im.shape[0])
    click_z = depth_im[y, x]
    click_z *= depth_scale
    click_x = (x-intrinsics_matrix[0, 2]) * \
        click_z/intrinsics_matrix[0, 0]
    click_y = (y-intrinsics_matrix[1, 2]) * \
        click_z/intrinsics_matrix[1, 1]
    if click_z == 0:
        raise Exception('Invalid pick point')
    # 3d point in camera coordinates
    point_3d = np.asarray([click_x, click_y, click_z])
    point_3d = np.append(point_3d, 1.0).reshape(4, 1)
    # Convert camera coordinates to world coordinates
    target_position = np.dot(pose_matrix, point_3d)
    target_position = target_position[0:3, 0]
    target_position[0] = - target_position[0]
    return target_position

In [15]:
depth_im = np.ones((729, 720)) * 2
pose_matrix = compute_pose(
                pos=[0, 2, 0],
                lookat=[0, 0, 0],
                up=[0, 0, 1])
pixel_to_3d(depth_im, 0, 0, pose_matrix)

array([-0.72000093,  0.        ,  0.72000093])

In [ ]:
#720 pixels = 0.720m
#1px = 1mm